## Main Objective is To find the relationship between car prices and body style, size and engine specs & To Find Do Body Style, Size and Engine Specs predict car prices?

I had cleaned the data that was retrieved from the database and done some data cleaning and feature engineering in our previous week assignment and saved in csv file named as "cars.csv" so, for this assignment I will be uisng that csv data to train our model.

I had generated 6 features from feature engineering process and the columns are 'fuel_efficiency', 'vehicle_volume', 'engine_power_density', 'performance_index', 'engine_performance','power2weight_ratio'

I will be using 3 features in my training phase, column 'performance_index' and 'engine_performance' as mandatory columns because they had good feature imporatance score and other one will be 'power2weight_ratio'.

In [31]:
import pandas as pd
import joblib
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from skopt import BayesSearchCV

In [32]:
df = pd.read_csv('cars.csv')


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 35 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   symboling             183 non-null    int64  
 1   normalized-losses     183 non-null    float64
 2   make                  183 non-null    int64  
 3   fuel-type             183 non-null    int64  
 4   aspiration            183 non-null    int64  
 5   num-of-doors          183 non-null    int64  
 6   body-style            183 non-null    int64  
 7   drive-wheels          183 non-null    int64  
 8   engine-location       183 non-null    int64  
 9   wheel-base            183 non-null    float64
 10  length                183 non-null    float64
 11  width                 183 non-null    float64
 12  height                183 non-null    float64
 13  curb-weight           183 non-null    int64  
 14  engine-type           183 non-null    int64  
 15  num-of-cylinders      1

In [34]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,price,num_doors,num_cylinders,engine_size_cc,power2weight_ratio,vehicle_volume,engine_power_density,fuel_efficiency,performance_index,engine_performance
0,2,164.0,0,1,0,0,3,1,0,99.8,...,13950.0,4,4,1786,0.043646,634816.956,0.935780,27.0,240.051348,255.0
1,2,164.0,0,1,0,0,3,0,0,99.4,...,17450.0,4,5,2228,0.040722,636734.832,0.845588,20.0,223.973088,184.0
2,2,161.0,0,1,0,1,3,1,0,99.8,...,15250.0,2,5,2228,0.043877,624189.969,0.808824,22.0,241.324292,187.0
3,1,158.0,0,1,0,0,3,1,0,105.8,...,17710.0,4,5,2228,0.038678,766364.046,0.808824,22.0,212.728551,187.0
4,1,161.0,0,1,0,0,4,1,0,105.8,...,18920.0,4,5,2228,0.037238,766364.046,0.808824,22.0,204.807041,187.0


In [35]:
df_copy = df

In [36]:
df_with_power2weight_ratio = df_copy.drop(columns=['fuel_efficiency', 'vehicle_volume', 'engine_power_density'])

### I will be defining different search space to find the optimal model and using the bayesian technique in iteration to find the optimal model

In [37]:

df = df_with_power2weight_ratio  # Ensure this dataset is preloaded

# Defining features and target variable
X = df.drop(columns=['price'])  
y = df['price']  

# Splitting the data into training (70%), validation (15%), and test (15%) sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Saving the validation set
val_set = pd.concat([X_val, y_val], axis=1)
val_set.to_csv("validation_set.csv", index=False)



The dataset is split into training (70%), validation (15%), and test (15%) sets.

Defining search space for optimization for different models 

In [38]:

models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'SVR': SVR(),
    'Ridge': Ridge()
}

search_spaces = {
    'RandomForest': {
        'n_estimators': (10, 200),
        'max_depth': (3, 30),
    },
    'SVR': {
        'C': (0.1, 1000, 'log-uniform'),
        'epsilon': (0.01, 1.0, 'uniform'),
    },
    'Ridge': {
        'alpha': (0.1, 10, 'log-uniform'),
    }
}


Performing Bayesian optimization

In [39]:
def bayesian_optimize(model, search_space):
    opt = BayesSearchCV(
        estimator=model,
        search_spaces=search_space,
        n_iter=30,
        cv=3,
        n_jobs=-1,
        random_state=42
    )
    opt.fit(X_train, y_train)
    return opt.best_estimator_

best_models = {name: bayesian_optimize(models[name], search_spaces[name]) for name in models}


Evaluating the best model

In [40]:
best_model_name = max(best_models, key=lambda name: r2_score(y_val, best_models[name].predict(X_val)))
best_model = best_models[best_model_name]
y_pred = best_model.predict(X_test)

print(f'Best Model: {best_model_name}')
print(f'R2 Score: {r2_score(y_test, y_pred)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')


Best Model: RandomForest
R2 Score: 0.9257043244883915
MAE: 1483.9225147776403


Saving the model

In [41]:
joblib.dump(best_model, f'best_{best_model_name.lower()}_model.pkl')

['best_randomforest_model.pkl']